# Setup

In [1]:
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.5f}'.format}, suppress = True)

from scipy.interpolate import interpn
from scipy.interpolate import RegularGridInterpolator

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib notebook

MEASFILENAME      = "measurements.txt"
MAGFIELDFILENAMES = [["r25s5", "r25s2_5", "r25s1_25"], ["r50s10", "r50s5", "r50s2_5"], ["r100s20", "r100s10", "r100s5"]]

for i in range(len(MAGFIELDFILENAMES)):
    for j in range(len(MAGFIELDFILENAMES[i])):
        MAGFIELDFILENAMES[i][j] = "magfields/" + MAGFIELDFILENAMES[i][j] + ".txt"

In [2]:
# For some reason this commands needs to be read twice to actually work
%matplotlib notebook

# File reading

In [3]:
# Reads a measurements file and returns an array with the measurement positions
def read_meas_file(filename):
    Z = []
    f = open(filename, "r")
    for line in f:
        meas = line.replace("\n", "").split(' ')
        Z.append(float(meas[0]))
    f.close()
    return Z

# Reads a BField file and returns a dictionary containing a set of vectors representing the field
def read_magfield_file(filename):
    B = {}
    f = open(filename, "r")

    for line in f:
        mag_meas = line.replace("\n", "").split(' ')
        for i in range(len(mag_meas)):
            mag_meas[i] = float(mag_meas[i])
        B[(mag_meas[0], mag_meas[1], mag_meas[2])] = (mag_meas[3], mag_meas[4], mag_meas[5])

    f.close()
    
    return B

# Magnetic Field Estimation

In [7]:
# Gets arrays of x, y, z, bx, by and bz in a specific plane on z0
def get_b(B, z0):
    b = ([], [], [], [], [], [])
    for key in B:
        if key[2] == z0:
            b[0].append(key[0])    # x
            b[1].append(key[1])    # y
            b[2].append(key[2])    # z
            b[3].append(B[key][0]) # bx
            b[4].append(B[key][1]) # by
            b[5].append(B[key][2]) # bz
    return b

# Creates and returns an interpolator for the magnetic field (bx,by,bz) at a given z plane (z0)
def get_z0_interpolator(B, z0):
    (x, y, z, bx, by, bz) = get_b(B, z0)
    
    # pre-process x and y
    grid_x = list(set(x))
    grid_y = list(set(y))
    grid_x.sort()
    grid_y.sort()
    grid_x = np.array(grid_x)
    grid_y = np.array(grid_y)
    
    # pre-process bx, by and bz
    grid_bx = []
    grid_by = []
    grid_bz = []

    for i in range(len(grid_x)):
        grid_bx.append([])
        grid_by.append([])
        grid_bz.append([])
        for j in range(len(grid_y)):
            grid_bx[i].append(bx[i*len(grid_x)+j])
            grid_by[i].append(by[i*len(grid_x)+j])
            grid_bz[i].append(bz[i*len(grid_x)+j])
    grid_bx = np.array(grid_bx)
    grid_by = np.array(grid_by)
    grid_bz = np.array(grid_bz)
    
    # obtain and return the interpolator
    bx_est = RegularGridInterpolator((grid_x, grid_y), grid_bx, method="linear", bounds_error=True)
    by_est = RegularGridInterpolator((grid_x, grid_y), grid_by, method="linear", bounds_error=True)
    bz_est = RegularGridInterpolator((grid_x, grid_y), grid_bz, method="linear", bounds_error=True)
    
    return (bx_est, by_est, bz_est)

# Plots a sample of the magnetic field at a cross section z0
def plot_b(B, z0):
    (x, y, z, bx, by, bz) = get_b(B, z0)

    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.quiver(x, y, z, bx, by, bz, length=3, normalize=True)

    plt.show()

# Tests

In [8]:
Z = read_meas_file(MEASFILENAME)
B = read_magfield_file(MAGFIELDFILENAMES[1][0])

In [12]:
(bx_est, by_est, bz_est) = get_z0_interpolator(B, Z[0])

for z in Z:
    (x, y, z, bx, by, bz) = get_b(B, z)
    print(bx)

[-0.008963, 0.002778, 0.007664, 0.006997, 0.003201, -0.001813, -0.006839, -0.010669, -0.011382, -0.006542, 0.005168, 0.001574, 0.009333, 0.010542, 0.007393, 0.002288, -0.003351, -0.008972, -0.01402, -0.01707, -0.015718, -0.007767, 0.010611, 0.01405, 0.011858, 0.006869, 0.001015, -0.004837, -0.010637, -0.016341, -0.021083, -0.022935, -0.019073, 0.017276, 0.016569, 0.011833, 0.005662, -0.00045, -0.006166, -0.0118, -0.01767, -0.023449, -0.027658, -0.027723, 0.021401, 0.01733, 0.010814, 0.00409, -0.001907, -0.007238, -0.01246, -0.018134, -0.024339, -0.030162, -0.0334, 0.023157, 0.016669, 0.009184, 0.002408, -0.003205, -0.007973, -0.01261, -0.017836, -0.023992, -0.030655, -0.036164, 0.023058, 0.015028, 0.007248, 0.00081, -0.004215, -0.008318, -0.012275, -0.016872, -0.022626, -0.029504, -0.036465, 0.021549, 0.012818, 0.005246, -0.000612, -0.004894, -0.008262, -0.01148, -0.015319, -0.020469, -0.027112, -0.034745, 0.019047, 0.010278, 0.003301, -0.001744, -0.005246, -0.007838, -0.010283, -0.013